In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [5]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(10, 10) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    
    return W1, b1, W2, b2,W3,b3

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2,W3,b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2,Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2,W3,b3 ,dW1, db1, dW2, db2,dW3,db3 ,alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3
    return W1, b1, W2, b2,W3,b3

In [6]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2,W3,b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2,Z3,A3 = forward_prop(W1, b1, W2, b2,W3,b3,X)
        dW1, db1, dW2, db2,dW3,db3 = backward_prop(Z1, A1, Z2, A2, Z3,A3,W1, W2,W3, X, Y)
        W1, b1, W2, b2,W3,b3 = update_params(W1, b1, W2, b2,W3,b3,dW1, db1, dW2, db2,dW3,db3 ,alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2,W3,b3
        

In [7]:
W1, b1, W2, b2,W3,b3 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[9 0 9 ... 0 0 0] [3 0 3 ... 8 5 4]
0.122
Iteration:  10
[6 7 3 ... 0 0 0] [3 0 3 ... 8 5 4]
0.1787560975609756
Iteration:  20
[6 7 3 ... 0 0 0] [3 0 3 ... 8 5 4]
0.2156341463414634
Iteration:  30
[6 6 3 ... 6 0 0] [3 0 3 ... 8 5 4]
0.24797560975609756
Iteration:  40
[6 0 3 ... 6 0 0] [3 0 3 ... 8 5 4]
0.27602439024390246
Iteration:  50
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.29478048780487803
Iteration:  60
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.3193658536585366
Iteration:  70
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.34585365853658534
Iteration:  80
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.3726829268292683
Iteration:  90
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.3993658536585366
Iteration:  100
[6 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.4235609756097561
Iteration:  110
[8 0 3 ... 8 0 0] [3 0 3 ... 8 5 4]
0.4476341463414634
Iteration:  120
[8 0 3 ... 8 0 4] [3 0 3 ... 8 5 4]
0.4730731707317073
Iteration:  130
[8 0 3 ... 8 0 4] [3 0 3 ... 8 5 4]
0.5014878048780488
Iteration:  140
[3 0 3 ...

In [8]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()